### Installation

After the installation is done, make sure to restart the session on Google Colab

In [ ]:
!pip install openagi

In [2]:
from openagi.agent import Admin
from openagi.llms.openai import OpenAIModel
from openagi.memory import Memory
from openagi.planner.task_decomposer import TaskPlanner
from openagi.worker import Worker

### Get your OpenAI API key

In [1]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI key:")

Enter your OpenAI key:··········


### Configure the Large Language Model

Documentation of another supported LLMs: [openagi.aiplanet.com/llms](https://openagi.aiplanet.com/components/llm)

Note: Its recommended to have OpenAI or Azure OpenAI for better quality

In [3]:
config = OpenAIModel.load_from_env_config()
llm = OpenAIModel(config=config)

### Multi-Agent Architecture

**Define Workers**

Workers are special type of classes, responsible for carrying out the tasks assigned by the class "Admin". They utilize tools such as internet search engines, LLMs, and document writers to perform their tasks. Additionally, they can determine which tools to use from a predefined set.

In [4]:
from openagi.actions.files import WriteFileAction, ReadFileAction
from openagi.actions.tools.ddg_search import DuckDuckGoSearch
from openagi.actions.tools.webloader import WebBaseContextTool

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Similarly to how a large task like writing a blog is decomposed into smaller steps such as researching, drafting, and publishing, the admin can define a large task and split it into smaller tasks that are then assigned to the workers.


The primary components, ``TaskWorker``, provide a structured way to define and execute tasks. ``The TaskWorker`` class specializes in executing specific tasks assigned by the planner.

In [5]:
feedback_collector = Worker(
        role="Customer Feedback Collector",
        instructions="Gather customer feedback specifically about AirPods Pro from various online platforms, including social media, review sites, and forums. Focus on identifying common themes and sentiments related to this product.",
        actions=[DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
    )

data_analyst = Worker(
    role="Data Analyst",
    instructions="Analyze the collected customer feedback data related to AirPods Pro to identify key trends, recurring issues, and overall customer sentiment. Use statistical tools to quantify the data and provide actionable insights.",
    actions=[ReadFileAction, DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
)

report_creator = Worker(
    role="Report Creator",
    instructions="Develop a comprehensive customer feedback analysis report based on the data analysis for AirPods Pro. Highlight key findings, trends, and recommendations for improving the product. Ensure the report is well-structured and visually appealing.",
    actions=[ReadFileAction, DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
)

### Define Admin Agent

Imagine Admin as the master task executor who is responsible for all the major configurations for the execution. From planning of tasks to execution, and defining the brain which is what LLM to use and whether or not to use memory.

Admin is the decision-maker that understand the specifications of tasks and execute them in a more human-like manner.

In [6]:
admin = Admin(
        actions=[DuckDuckGoSearch],
        planner=TaskPlanner(human_intervene=False),
        memory=Memory(),
        llm=llm,
    )
admin.assign_workers([feedback_collector, data_analyst, report_creator])

## Run the Admin

Check your directory, all the Workers tasks results is saved inside ``Log files``.

Once the execution is completed, it will save the results of the Report Creator Agent inside the txt file.

In [ ]:
res = admin.run(
        query="Create a customer feedback analysis report for AirPods Pro.",
        description="Collect and analyze customer feedback specifically for AirPods Pro from multiple online sources. Identify common themes, recurring issues, and overall customer sentiment. Develop a comprehensive report that provides actionable insights and recommendations for improving the AirPods Pro. Ensure the report is detailed, well-organized, and visually appealing.",
    )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




Tasks: [{'task_name': 'Collect Feedback Data', 'description': 'Use DuckDuckGoSearch to search for customer feedback about AirPods Pro from various online platforms like social media, review sites, and forums. Focus on identifying common themes and sentiments related to this product. Gather URLs and relevant data from the search results.', 'worker_id': '4da8ee85733e40f6bcc949b174f2ac77'}, {'task_name': 'Extract Web Content', 'description': 'Use WebBaseContextTool to load and summarize the content from the URLs collected in the previous task. Extract actual context, metadata and page content as a context string from each URL.', 'worker_id': '4da8ee85733e40f6bcc949b174f2ac77'}, {'task_name': 'Write Feedback Data to File', 'description': 'Use WriteFileAction to write the content extracted in the previous task to a file. This file will store all the summarized customer feedback for later analysis.', 'worker_id': '4da8ee85733e40f6bcc949b174f2ac77'}, {'task_name': 'Analyze Feedback Data', '

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 40.5MiB/s]
ERROR:root:Error running action: 'title'


In [11]:
!ls

AirPodsPro_AnalysisResults.txt	 f4c0bd8917724eabae857bc81ca91d90  sample_data
AirPodsPro_CustomerFeedback.txt  logs


In [12]:
with open("AirPodsPro_CustomerFeedback.txt","r") as f:
    content = f.read()
    print(content)

The Apple AirPods Pro has been praised for its superior noise cancellation abilities, enhanced audio performance, and the comfort of the in-ear fit. The charging case provides an additional 19 to 20 hours of battery life, although performance can vary with volume levels. Features such as ANC (Active Noise Cancellation) and Transparency Modes are effective in reducing ambient noise while allowing important environmental sounds, facilitating natural conversations. The audio performance is robust, featuring strong bass and clear highs, marking a significant improvement over the standard AirPods model. Customer feedback from review sites like Consumer Reports, The Verge, Engadget, and PCMag highlights praise for these improvements but also points out issues such as short battery life and reduced functionality on non-Apple devices.
